# RISE Camp 2018: Introduction to Flor!


Welcome to RISE Camp 2018! Flor is a system for managing workflow development within the machine learning lifecycle. This tool enables data scientists to describe ML workflows as directed acyclic graphs (DAGs) of Actions, Artifacts, or Literals and to experiment with different configurations quickly by running multi-trial experiments. 

The purpose of this notebook is to help you use Flor in order to naviagte through different parts of the data science lifecycle.

As you work through this notebook, you will learn:

* How to define/use experiments, literals, artifacts and actions.
* How to run experiments with different congigurations.
* Compare models with other past versions in order to select the best model.

We will be working with a ratings dataset. Our goal is to predict whether a movie review is positive or negative based on its text.

**Data science is a collaborative activity - we encourage you to work with those around you and ask questions!**

## Imports

In [1]:
#General imports
import pandas as pd
import numpy as np
import nltk
import json
import time
import random
import scipy.sparse
import flor

#Pre-processing imports
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

#Model training and testing imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import average_precision_score, recall_score, precision_recall_fscore_support

In [2]:
# If the notebook name has not already been set, you are able to set the name in code. 
flor.setNotebookName('tutorial.ipynb')

In [3]:
%load_ext autoreload
%autoreload 2

## Preprocessing Data

Before building our model, we will define a pipeline to pre-process our text data. We have used the following techniques to pre-process the text reviews:
* Removal of Stop Words
* Stemming (reducing inflected words to their stem)
* Lemmatization (group together inflected forms of a words)


In [8]:
# Run me
from preprocess import preprocessing

## Train Test Split

We have created a function to split our data into train/test sets. Since we would like this to be a Flor Function, we must wrap it with the @flor.func decorator so it is able to be referenced by Flor actions. **Please navigate to the florfunctions.py file and wrap the `traintest_split` function with the Flor decorator.**

In [ ]:
# Run me
from florfunctions import traintest_split

## Model Training and Evaluation

Now, we can create a Flor function to train and evaluate a model to classify reviews into rating buckets. **Please navigate to the florfunctions.py file and complete the `train_test` function; fill in the Random Forest model with an n_estimators parameter of 5.**

In [ ]:
from florfunctions import train_test

## Experiment Setup

Finally, we will now define our Flor experiment using the Flor functions we created above. Proceed through the following cells and run the experiment.

In [9]:
ex = flor.Experiment('risecamp_demo').__enter__()

In [10]:
data = ex.artifact('data.json', 'data_loc')
do_preproc = ex.action(preprocessing, [data,])
data_x = ex.artifact('data_clean_X.json', 'intermediate_X', parent=do_preproc)
data_y = ex.artifact('data_clean_y.json', 'intermediate_y', parent=do_preproc)
ex.__exit__()

In [11]:
data_x.pull(utag='second_preproc')

Data Preprocessing
--- 197.00618505477905 seconds ---


Next, we need to load the data we want to analyze. We can load the data by creating **artifacts**, which are pointers to data we want. In this case, we have already generated cleaned data from a previous experiment run; we can retrieve the cleaned data by referencing the tag of the particular run.

In [ ]:
# Defines preprocessing action and resulting intermediary artifacts
#TODO: double check syntax
data_x = ex.artifact('data_clean_X.json', 'intermediate_X', utag="first")
data_y = ex.artifact('data_clean_y.json', 'intermediate_y', utag="first")

Now that we have our data points, we need to perform a train/test split. Using the `traintest_split` function we imported earlier, let's create a flor action as well as the intermediary artifacts generated by the split.

In [ ]:
#traintest_split is the function to run, data_x and data_y are arguments
do_split = ex.action(traintest_split, [data_x, data_y])

In [ ]:
#artifacts have a pointer (filename), internal name, and (optional) parent
X_train = ex.artifact('x_train.npz', 'X_train', do_split)
X_test = ex.artifact('x_test.npz', 'X_test', do_split)
y_train = ex.artifact('y_train.json', 'y_train', do_split)
y_test = ex.artifact('y_test.json', 'y_test', do_split)

Lastly, we can specify the hyperparameter with a **literal**, an explicit value stored in Flor, and create an action for our `train_test` function and an artifact for our result. We can wrap up the experiment and close it with `__exit__()`.

In [ ]:
hyperparameter = ex.literal(v = 5, name="hyperparameters")
#Define the model training and evaluation action and final artifacts
do_test = ex.action(train_test, [X_train, X_test, y_train, y_test, hyperparameter])
report = ex.artifact('report.csv', 'report', do_test)

In [ ]:
flor.Experiment("risecamp_demo").__exit__()

## Pull

Finally, we are ready to run the experiment! We can do so by running `pull()` on our output artifacts. Before doing this, however, it is helpful to use `plot()` to generate a florplan, a graph representation of the artifact's lineage.

In [ ]:
report.plot()

In [ ]:
#Run the experiment
report.pull("first_pull")

## Hyperparameter Tuning

Flor makes it convenient to run models using different hyperparameters and track the results. In the `train_test` we created function, notice that we pass in `hyperparameters` in addition to the train and test data. These hyperparameters will allow us to tune our model and track results with ease; let's define them in our experiment setup.

Notice that the Random Forest Classifier contains `n_estimators` as a hyperparameter. We would like to tune this hyperparameter and track model performance. In order to specify the hyperparameters, we must make a `literalForEach` within our experiment. **Fill in the `literalForEach` with values 5, 50 and 75 within the experiment below.**

In [ ]:
#Note: We have copied the same experiment below for convenience.
#We can also create flor experiments using a context manager.

# Create a context manager for the experiment and is named 'risecamp_demo'
with flor.Experiment('risecamp_demo') as ex:
    ex.groundClient('git') #use "git" from grit and "ground" for ground

    # Defines artifacts
    data_x = ex.artifact('data_clean_X.json', 'intermediate_X', utag='first')
    data_y = ex.artifact('data_clean_y.json', 'intermediate_y', utag='first')

    #Define split action and resulting output artifacts
    do_split = ex.action(traintest_split, [data_x, data_y])
    X_train = ex.artifact('x_train.npz', 'X_train', do_split)
    X_test = ex.artifact('x_test.npz', 'X_test', do_split)
    y_train = ex.artifact('y_train.json', 'y_train', do_split)
    y_test = ex.artifact('y_test.json', 'y_test', do_split)

    #Define the hyperparameters for the models
    random_forest_Nestimators = ex.literal(v = 5, name="hyperparameters")
#     random_forest_Nestimators = ex.literalForEach(v=[5, 50, 75], name="hyperparameters", default=50) #SOLUTION

    #Define the model training and evaluation action and final artifacts
    do_test = ex.action(train_test, [X_train, X_test, y_train, y_test, random_forest_Nestimators])
    report = ex.artifact('report.csv', 'report', do_test)


In [ ]:
report.plot()

In [ ]:
#Run the experiment
report.pull(utag="hyperparameter_tuning")

## Peeking at Results

After running the model with different hyperparameters above, we are able to peek at our output artifact, containing precision and recall metrics for the different models. Run the following cell - **which hyperparameter yields the best model?**

In [ ]:
#Run me
report.peek() #Depends on dataframe

## Building a Better Model

Now that you have some experience using flor, let's try using a different model to see if we can improve the results. Some of the classifiers we recommend trying are the Multilayer Perceptron Classifier, Naive Bayes Classifier, and K-neighbors Classifier.

After implementing your model of choice in the `train_test` function in **florfunctions.py**, run the cells below to reimport the function and run the experiment.

In [ ]:
from florfunctions import train_test

In [ ]:
#Note: We have copied the same experiment below for convenience.
#We can also create flor experiments using a context manager.

# Create a context manager for the experiment and is named 'risecamp_demo'
with flor.Experiment('risecamp_demo') as ex:
    ex.groundClient('git') #use "git" from grit and "ground" for ground

    # Defines artifacts
    data_x = ex.artifact('data_clean_X.json', 'intermediate_X', utag='first')
    data_y = ex.artifact('data_clean_y.json', 'intermediate_y', utag='first')

    #Define split action and resulting output artifacts
    do_split = ex.action(traintest_split, [data_x, data_y])
    X_train = ex.artifact('x_train.npz', 'X_train', do_split)
    X_test = ex.artifact('x_test.npz', 'X_test', do_split)
    y_train = ex.artifact('y_train.json', 'y_train', do_split)
    y_test = ex.artifact('y_test.json', 'y_test', do_split)

    #Define the hyperparameters for the models
    #hyperparameter = ex.literal(v = 5, name="hyperparameters")
    random_forest_Nestimators = ex.literalForEach(v=[5, 50, 75], name="hyperparameters", default=50) #SOLUTION
    #MLP_hidden_layer_size = ex.literalForEach(v=[(1, ), (2, ), (3, )], name="hyperparameters", default=(2, ))

    #Define the model training and evaluation action and final artifacts
    #change to MLP_hidden_layer_size 
    do_test = ex.action(train_test, [X_train, X_test, y_train, y_test, random_forest_Nestimators])
    #for naive bayes
    #do_test = ex.action(train_test, [X_train, X_test, y_train, y_test])
    report = ex.artifact('report.csv', 'report', do_test)


In [ ]:
report.pull(utag="improved_model")

In [ ]:
report.peek()

## Data Mishaps

It turns out, the data that we have been working had not been cleaned optimally to begin with. In fact, we can observe the exact cleaning process in the `pre-processing` function within florfunctions.py. We can re-clean raw data by adding another flor action and intermediate artifacts. Fortunately, however, we already ran an experiment with more optimal preprocessing. We can checkout the artifacts by using `utag = 'second'`.

In [ ]:
data_x = ex.artifact('data_clean_X.json', 'intermediate_X', utag='first')
data_y = ex.artifact('data_clean_y.json', 'intermediate_y', utag='first')


Lets see how the new data impacts our results.

In [ ]:
#Note: We have copied the same experiment below for convenience.

# Create a context manager for the experiment and is named 'risecamp_demo'
with flor.Experiment('risecamp_demo') as ex:
    ex.groundClient('git') #use "git" from grit and "ground" for ground

    # Defines artifacts
    data_x = ex.artifact('data_clean_X.json', 'intermediate_X', utag='second')
    data_y = ex.artifact('data_clean_y.json', 'intermediate_y', utag='second')

    #Define split action and resulting output artifacts
    do_split = ex.action(traintest_split, [data_x, data_y])
    X_train = ex.artifact('x_train.npz', 'X_train', do_split)
    X_test = ex.artifact('x_test.npz', 'X_test', do_split)
    y_train = ex.artifact('y_train.json', 'y_train', do_split)
    y_test = ex.artifact('y_test.json', 'y_test', do_split)

    #Define the hyperparameters for the models
    #hyperparameter = ex.literal(v = 5, name="hyperparameters")
    random_forest_Nestimators = ex.literalForEach(v=[5, 50, 75], name="hyperparameters", default=50) #SOLUTION
    #MLP_hidden_layer_size = ex.literalForEach(v=[(1, ), (2, ), (3, )], name="hyperparameters", default=(2, ))

    #Define the model training and evaluation action and final artifacts
    #change to MLP_hidden_layer_size 
    do_test = ex.action(train_test, [X_train, X_test, y_train, y_test, random_forest_Nestimators])
    #for naive bayes
    #do_test = ex.action(train_test, [X_train, X_test, y_train, y_test])
    report = ex.artifact('report.csv', 'report', do_test)

In [ ]:
report.pull(utag="better_data")